# Preprossesering med Python

Før vi kan begynne å jobbe med dataene må vi ta en titt på dem og se at alt ser ok ut, og rense opp der det ikke gjøre det. Vi må sjekke at alt har blitt lastet inn skikkelig, at alle kolonner er av riktig type og se om det er noen kolonner som mangler for mye informasjon til at vi får brukt dem. Denne første fasen kalles preprossesering og er veldig viktig for å bli kjent med datasettet og få god datakvalitet for de påfølgende analysene. 

Vi starter med å importere pakkene vi trenger og å laste inn dataene over ledige stillinger fra 2002 til 2017:

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sci
import re

Det er lurt å se på filene før vi importerer dem. Da ser vi at '\*' brukes som tegn for manglende data, og kan 'si' det til read_csv-funksjonen. Litt avhengig av innstillingene hos deg så kan csv'er også være semikolonseparerte i stedet for kommaseparerte. Da må man lenge til argumentet sep=';'. Du trenger ikke tenke på dette så lenge du kjører via Notebooken, men det er greit å være klar over hvis du skal lese inn en fil fra din egen maskin.

In [ ]:
period = [2002, 2017]
jobs = pd.DataFrame()

for year in range(period[0], period[1]+1):
    file = 'open-data/data/raw/Ledigestillinger_'+str(year)+'.csv'
    yearly = pd.read_csv(file, na_values='*')
    jobs = ledige.append(yearly, ignore_index=True)


Når vi prøver å laste inn dataene så får vi en feilmelding - kolonne 0 og 1 har blandede datatyper. Ser vi i filene så ser det ut som begge kolonnene skal være numeriske, så det har nok sneket seg inn litt tekst innimellom tallene. Hvis vi legger til 'print(year)' i loopen så ser vi at problemet stammer fra 2015-filen.

Et greit sted å starte å nøste er å se på hvilke typer kolonnene fikk når vi lastet dem inn, ettersom dette velges automatisk:

In [ ]:
jobs.dtype

Vi ser at kolonne 0 og 1 har fått type 'object', som vil si string. Vi kan også se hvordan de første radene ser ut for å få en følelse av dataene. For å få med alle kolonner må vi endre en instilling, ellers vises bare de første og siste radene.

In [ ]:
# Show all 23 columns (truncates without)
pd.set_option('display.max_columns', jobs.shape[1]) 

jobs.head()

En annen ting som er lurt å sjekke, spesielt når vi har fått en slik feilmelding, er at alle årene har blitt lastet inn. For å få det gruppert på år i stedet for måned bruker jeg integer division, som alltid runder nedover:

In [ ]:
#yyyymm -> yyyy
sci.itemfreq(jobs.statistikk_aar_mnd//100)

Her ser vi at alle år er med. Da må vi gå til verks med å finne ut hvor det går galt i 2015-filen. Jeg fant en smart liten funksjon her (legg inn link) som jeg har modifisert til å finne ut om et element inneholder noe annet enn tall:

In [ ]:
def special_match(strg, search=re.compile(r'[^0-9]').search):
    return bool(search(strg))

#checks strings one-by-one. Adding loop to go through a whole column:
jobs[[special_match(str(nr)) for nr in jobs.stillingsnummer]]

Her ser vi at både kolonne 0 og 1 har en string i rad nr 570062, og ganske riktig så kommer den fra 2015. Det ser ut som at enheten har blitt skrevet inn som 'aetat' i stedet for koden. Men dette er ikke *egentlig* numeriske data - verdiene betyr ikke noe i seg selv, og rekkefølgen har ingenting å si. Det er egentlig kategoriske data. Vi kan la kolonnene stå som string, eller vi kan gjøre dem om til 'categorical', så Python kjenner dem igjen som det. Før vi bestemmer oss kan vi sjekke hvor mange grupper det ville blitt i så fall:

In [ ]:
np.unique(jobs.nav_enhet_kode).size
np.unique(jobs.stillingsnummer).size

Det er 704 ulike nav-enheter og over 2.5 millioner unike stillingsnummer, nesten like mange som det er rader i dataframen vår. For stillingsnummerene er det derfor kanskje like greit å bare la de være strings, mens det gir mening å tenke på enhetskodene som grupperinger.

In [ ]:
jobs.nav_enhet_kode = jobs.nav_enhet_kode.astype('category')
jobs.dtypes  # check if it worked

Det er flere kolonner vi kan vurdere å gjøre om til category, men vi kan la de være det de er inntil videre. Vi kan alltid endre det senere hvis behovet skulle dukke opp. (fikse fylke/ kommune med en gang?)

En annen ting vi ser i utskriften over er at datoene er lagret som tekst. Disse kan vi konvertere til datoformat. Pandas funksjon to_datetime virker bare på endimensjonal input, så jeg bruker apply for å kunne gjøre det på begge dato-kolonnene samtidig. Datoene blir nå printet ut i yyyy-mm-dd format.

In [ ]:
jobs.iloc[:, 2:4] = jobs.iloc[:, 2:4].apply(pd.to_datetime, errors='coerce', format='%d.%m.%Y')
jobs.iloc[:, 2:4].head()

Det er en del feilpunching i kolonnen 'sistepubl_dato' som gjør at vi får feilmeldinger hvis vi årøver å konvertere uten å bruke errors-argumentet. 'Coerce' vil si at alle felter som genererer feilmeldinger blir gjort om til 'NaT' - Not a Time. 

Vi kan også skrive ut de mest vanlige numeriske oppsummeringene, selv om det i vårt tilfelle ikke er relevant for store deler av kolonnene - ops ser kommune med feil verdier, fiks

De andre feltene ser ut til å ha riktig typer. La oss derfor undersøke hvor mye data som mangler i hver kolonne: